In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
data = pd.read_excel(r"C:\Users\hinaj\OneDrive\Documents\Project datasets\online_retail_II.xlsx", sheet_name=0)

In [ ]:
data.head()

In [ ]:
## CREATING COPY OF THE ORIGINAL DATASET

df = data.copy()
df

In [ ]:
## GENERAL DESCRIPTION OF DATASET

df.info()

In [ ]:
df.describe()

In [ ]:
df.describe(include='O')

CHECKING ALL COLUMNS OF THE DATASET

In [ ]:
## LOOKING AT MISSING CUSTOMER ID

df[df['Customer ID'].isna()]

In [ ]:
## A LOOK AT NEGATIVE QUANTITIES 

df[df['Quantity'] < 0].head(10)

## Most of the Data does have Customer ID and looks valid 

In [ ]:
## A LOOK AT NEGATIVE PRICE

df[df['Price'] < 0]

## These are data with bad debts

In [ ]:
## LOOKING AT CANCELLED INVOICE (STARTS WITH LETTER 'C' IN THE BEGINNING OF INVOICE)

df['Invoice'] = df['Invoice'].astype('str')
df[df['Invoice'].str.match('^\\d{6}$') == False]

In [ ]:
## CHECKING IF INVOICES CONTAIN OTHER ALPHABETS

df['Invoice'].str.replace('[0-9]', '', regex=True).unique()

In [ ]:
df[df['Invoice'].str.startswith('A')]

## Invoice starting with A specifies bad debt accounts of the store

In [ ]:
## CHECKING STOCKCODE COLUMN (5digits + Alphabet)

df['StockCode'].astype('str')
df[(df['StockCode'].str.match('^\\d{5}$') == False) & (df['StockCode'].str.match('^\\d{5}[a-zA-Z]+$') == False)] 

In [ ]:
## LOOKING AT DISTINCT STOCK CODE VALUES

df[(df['StockCode'].str.match('^\\d{5}$') == False) & (df['StockCode'].str.match('^\\d{5}[a-zA-Z]+$') == False)]['StockCode'].unique() 

## These stock codes specify different charges incurred by store which do not hold any significance

In [ ]:
## LOOKING AT COUNTRY COLUMN 

df['Country'].unique()

In [ ]:
## LOOKING EIRE COUNTRY

df[df['Country'].str.contains('EIRE')]

## Data does not contain any problem

In [ ]:
## LOOKING AT UNSPECIFIED COUNTRY

df[df['Country'].str.contains('Unspecified')]

## Data does not contain any problem

STARTING DATA CLEANING

In [ ]:
## REMOVING UNWANTED VALUES FROM INVOICE COLUMN 

df['Invoice'] = df['Invoice'].astype('str')

mask = (
    df['Invoice'].str.match('^\\d{6}$') == True
)

df = df[mask]
df

In [ ]:
## REMOVING UNWANTED VALUES FROM STOCK CODE COLUMN

df['StockCode'] = df['StockCode'].astype('str')

mask = (
    (df['StockCode'].str.match('^\\d{5}$') == True )
    | (df['StockCode'].str.match('^\\d{5}[a-zA-Z]+$') == True)
    | (df['StockCode'].str.match('^PADS$') == True)
)

df = df[mask]
df

In [ ]:
df.describe()

In [ ]:
## DROPIING NULLS FROM CUSTOMER ID 

df.dropna(subset='Customer ID', inplace=True)

In [ ]:
df.describe()

In [ ]:
## LOOKING AT 0 PRICE VALUES 

len(df[df['Price'] == 0])

## 28 values with 0 price 

In [25]:
## REMOVING 0 PRICE VALUES 

df = df[df['Price'] > 0]

In [ ]:
df.describe()

In [ ]:
## CHECKING NUMBER OF DROPPED DATASET

len(df)/len(data)

## 77% of data is compatable 

DROPPED 23% OF UNWANTED DATASET

ANALYSIS 

In [ ]:
## CREATING SALES ANALYSIS 

df['Sales'] = df['Price'] * df['Quantity']
df

In [ ]:
aggregated_df = df.groupby(by='Customer ID', as_index=False) \
    .agg(
        MonetaryValue=('Sales', 'sum'),
        Frequency=('Invoice', 'nunique'),
        LastInvoiceDate=('InvoiceDate', 'max')
    )

aggregated_df

In [ ]:
max_invoice_date = aggregated_df['LastInvoiceDate'].max()
aggregated_df['Recency'] = (max_invoice_date - aggregated_df['LastInvoiceDate']).dt.days
aggregated_df

## Last date (2012-12-09)

In [ ]:
## CHECKING OUTLIERS IN THE DATASET (THROUGH BOX PLOT)

plt.figure(figsize=(15,5))

plt.subplot(1,3,1)
sns.boxplot(data=aggregated_df['MonetaryValue'], color='blue')
plt.title('Monetary Value Plot')
plt.xlabel('Monetary value')

plt.subplot(1,3,2)
sns.boxplot(data=aggregated_df['Frequency'], color='green')
plt.title('Frequency Value Plot')
plt.xlabel('Frequency value')

plt.subplot(1,3,3)
sns.boxplot(data=aggregated_df['Recency'], color='pink')
plt.title('Recency Value Plot')
plt.xlabel('Recency value')

plt.tight_layout()
plt.show()

## Monetary value and frequency value has lots of outliers

SEPERATING OUTLIERS

In [ ]:
## OUTLIERS IN MONETARY VALUE

M_Q1 = aggregated_df['MonetaryValue'].quantile(0.25)
M_Q3 = aggregated_df['MonetaryValue'].quantile(0.75)

M_IQR = (M_Q3 - M_Q1)

Monetary_outliers_df = aggregated_df[(aggregated_df['MonetaryValue'] > (M_Q3 + 1.5*M_IQR)) | (aggregated_df['MonetaryValue'] < (M_Q1 - 1.5*M_IQR))].copy()
Monetary_outliers_df

In [ ]:
Monetary_outliers_df.describe()

In [ ]:
## OUTLIERS IN FREQUENCY VALUES

F_Q1 = aggregated_df['Frequency'].quantile(0.25)
F_Q3 = aggregated_df['Frequency'].quantile(0.75)

F_IQR = (F_Q3 - F_Q1)

Frequency_outliers_df = aggregated_df[(aggregated_df['Frequency'] > (F_Q3 + 1.5*F_IQR)) | (aggregated_df['Frequency'] < (F_Q1 - 1.5*F_IQR))].copy()
Frequency_outliers_df

In [ ]:
Frequency_outliers_df.describe()

In [ ]:
## CREATING NEW DATASET WITHOUT OUTLIERS 

Non_outliers_df = aggregated_df[(~aggregated_df.index.isin(Monetary_outliers_df.index)) & (~aggregated_df.index.isin(Frequency_outliers_df.index))]
Non_outliers_df.describe()

In [ ]:
## PLOTTING NON OUTLIERS DATASET

fig = plt.figure(figsize=(7,7))

a = fig.add_subplot(projection ='3d')
scatter = a.scatter(Non_outliers_df['MonetaryValue'], Non_outliers_df['Frequency'], Non_outliers_df['Recency'])

a.set_xlabel('Monetary value')
a.set_ylabel('Frequency')
a.set_zlabel('Recency')

a.set_title('3D plot of Customer data')

plt.show()

In [38]:
## IMPORTING LIBRARIES FOR CLUSTERING ANALYSIS 

from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler

pd.options.display.float_format = '{:20.2f}'.format

In [ ]:
scalar = StandardScaler()

scaled_data = scalar.fit_transform(Non_outliers_df[['MonetaryValue', 'Frequency', 'Recency']])
scaled_data

In [ ]:
scaled_df = pd.DataFrame(scaled_data, index=Non_outliers_df.index, columns=('Monetary values', 'Frequency', 'Recency'))
scaled_df

In [ ]:
## PLOTTING WITH STANDARDIZE VALUES 

fig = plt.figure(figsize=(7,7))

a = fig.add_subplot(projection ='3d')
scatter = a.scatter(scaled_df['Monetary values'], scaled_df['Frequency'], scaled_df['Recency'])

a.set_xlabel('Monetary value')
a.set_ylabel('Frequency')
a.set_zlabel('Recency')

a.set_title('3D plot of Customer data')

plt.show()

KMEANS CLUSTERING 

In [ ]:
## LOOKING FOR SUITABLE NUMBER OF CLUSTERS

max_k = 12 
inertia = []
k_values = range(2, max_k+1)

for k in k_values:
    
    k_means = KMeans(n_clusters=k, random_state=42, max_iter=1000)
    k_means.fit_predict(scaled_df)
    inertia.append(k_means.inertia_)

plt.figure(figsize=(14,6))
plt.plot(k_values, inertia, marker='o')

plt.title('kMeans inertia values')
plt.xlabel('Number of clusters (K)')
plt.ylabel('Inertia')
plt.xticks(k_values)
plt.grid(True)

plt.show()

## 4 Number of clusters looked appropriate for this dataset

In [ ]:
## SETTING CLUSTERS IN DATAFRAME

k_means = KMeans(n_clusters=4, random_state=42, max_iter=1000)
cluster_labels = k_means.fit_predict(scaled_df)

cluster_labels

In [ ]:
Non_outliers_df['clusters'] = cluster_labels
Non_outliers_df

In [ ]:
## PLOTTING CLUSTER 

cluster_colors = {0: '#1f77b4', #Blue
                  1:'#ff7f0e',   #Orange
                  2: '#2ca02c', #Green
                  3: '#d62728'} #Red

colors = Non_outliers_df['clusters'].map(cluster_colors)

fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(projection='3d')

scatter = ax.scatter(Non_outliers_df['MonetaryValue'], 
                     Non_outliers_df['Frequency'], 
                     Non_outliers_df['Recency'], 
                     c=colors, # Use mapped solid colors 
                     marker='o')

ax.set_xlabel('Monetary Value')
ax.set_ylabel('Frequency')
ax.set_zlabel('Recency')

ax.set_title('3D Scatter Plot of Customer Data by cluster')

plt.show()

CLUSTER ANALYSIS 

Cluster 0 (Blue) = Regular Buyers with moderate monetary values and high frequency = 'Promotion'
Cluster 1 (Orange) = Old Buyers with moderate frequency but have not purchased since long = 'Re-Engage'
Cluster 2 (Green) = New Buyers with less monetary value = 'Engagement'
Cluster 3 (Red) = Buyers with very high monetary value and high frequency = 'Loyal'


In [ ]:
## OUTLIERS CLUSTERS 

overlap_indices = Monetary_outliers_df.index.intersection(Frequency_outliers_df.index)
overlap_indices

Monetary_only_outliers_df = Monetary_outliers_df.drop(overlap_indices)
Frequency_only_outliers_df = Frequency_outliers_df.drop(overlap_indices)
Monetary_and_Frequency_outliers = Monetary_outliers_df.loc[overlap_indices]

Monetary_only_outliers_df['clusters'] = -1
Frequency_only_outliers_df['clusters'] = -2
Monetary_and_Frequency_outliers['clusters'] = -3

Outliers_df = pd.concat([Monetary_only_outliers_df, Frequency_only_outliers_df, Monetary_and_Frequency_outliers])
Outliers_df

In [ ]:
## PLOTTING OUTLIER CLUSTERS

cluster_colors = {-1: '#9467bd',
                  -2:'#8c564b',   
                  -3: '#e377c2'} 
                  

colors = Outliers_df['clusters'].map(cluster_colors)

fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(projection='3d')

scatter = ax.scatter(Outliers_df['MonetaryValue'], 
                     Outliers_df['Frequency'], 
                     Outliers_df['Recency'], 
                     c=colors, # Use mapped solid colors 
                     marker='o')

ax.set_xlabel('MonetaryValue')
ax.set_ylabel('Frequency')
ax.set_zlabel('Recency')

ax.set_title('3D Scatter Plot of Outlier Data by cluster')

plt.show()

OUTLIERS CLUSTER ANALYSIS 

Cluster -1 = High spendings but infrequent buyers = 'GetBack'
Cluster -2 = High frequent but less monetary values = 'NeedDeals'
Cluster -3 = High monetary value and high frequency = 'VIPs'

In [58]:
## DEFINING CUTSOMERS IN DATASET 

cluster_labels = {
    0:'RETAIN',
    1:'RE-ENGAGE',
    2:'NURTURE',
    3:'REWARD',
    -1:'PAMPER',
    -2:'UPSELL',
    -3:'DELIGHT'
}

In [ ]:
Full_cluster_df = pd.concat([Non_outliers_df, Outliers_df])
Full_cluster_df

In [ ]:
Full_cluster_df['ClusterLabels'] = Full_cluster_df['clusters'].map(cluster_labels)
Full_cluster_df

In [ ]:
Full_cluster_df.groupby(by='ClusterLabels').count()